In [5]:
# Data
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import impl.layer as l

# Dataset preparation and pre-processing
mnist = input_data.read_data_sets('data/MNIST_data/', one_hot=False)

X_train, y_train = mnist.train.images, mnist.train.labels
X_val, y_val = mnist.validation.images, mnist.validation.labels
X_test, y_test = mnist.test.images, mnist.test.labels
# y_test.shape, y_val.shape, y_train.shape
# X_train.shape, X_train.dtype, X_val.shape, X_val.dtype, X_test.shape, X_test.dtype

Extracting data/MNIST_data/train-images-idx3-ubyte.gz
Extracting data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting data/MNIST_data/t10k-labels-idx1-ubyte.gz


In [6]:
# Pre-processing: normalizing
def normalize(X):
    # max scale for images 255= 2**8= 8 bit grayscale for each channel
    return (X - X.mean(axis=0)) #/ X.std(axis=0)

X_train, X_val, X_test = normalize(X=X_train), normalize(X=X_val), normalize(X=X_test)

In [ ]:
# Model
import impl.layer as l # or from impl.layer import *
from impl.loss import * # import all functions from impl.loss file # import impl.loss as loss_func
from sklearn.utils import shuffle as skshuffle

class FFNN:

    def __init__(self, D, C, H, L):
        self.L = L # number of layers or depth
        self.losses = {'train':[], 'valid':[], 'valid_acc':[]}
        
        # The adaptive/learnable/updatable random feedforward
        self.model = []
        self.W_fixed = []
        self.grads = []
        self.ys_prev = []
        low, high = -1, 1
        
        # Input layer: weights/ biases
        m = dict(W=np.random.uniform(size=(D, H), low=low, high=high) / np.sqrt(D / 2.), b=np.zeros((1, H)))
        self.model.append(m)
        # Fixed feedback weight
        m = np.random.uniform(size=(D, H), low=low, high=high) / np.sqrt(D / 2.)
        self.W_fixed.append(m)
        # Input layer: gradients
        self.grads.append({key: np.zeros_like(val) for key, val in self.model[0].items()})
        # Previous output layer
        self.ys_prev.append(0.0)

        # Hidden layers: weights/ biases
        m_L = []
        for _ in range(L):
            m = dict(W=np.random.uniform(size=(H, H), low=low, high=high) / np.sqrt(H / 2.), b=np.zeros((1, H)))
            m_L.append(m)
        self.model.append(m_L)
        # Fixed feedback weight
        m_L = []
        for _ in range(L):
            m = np.random.uniform(size=(H, H), low=low, high=high) / np.sqrt(H / 2.)
            m_L.append(m)
        self.W_fixed.append(m_L)
        # Hidden layer: gradients
        grad_L = []
        for _ in range(L):
            grad_L.append({key: np.zeros_like(val) for key, val in self.model[1][0].items()})
        self.grads.append(grad_L)
        # Previous output layer
        ys_prev_L = []
        for _ in range(L):
            ys_prev_L.append(0.0)
        self.ys_prev.append(ys_prev_L)
        
        # Output layer: weights/ biases
        m = dict(W=np.random.uniform(size=(H, C), low=low, high=high) / np.sqrt(H / 2.), b=np.zeros((1, C)))
        self.model.append(m)
        # Fixed feedback weight
        m = np.random.uniform(size=(H, C), low=low, high=high) / np.sqrt(H / 2.)
        self.W_fixed.append(m)
        # Outout layer: gradients
        self.grads.append({key: np.zeros_like(val) for key, val in self.model[2].items()})
        # Previous output layer
        self.ys_prev.append(0.0)
        
    def fc_forward(self, X, W, b):
        out = (X @ W) + b
        cache = (W, X)
        return out, cache

    def fc_backward(self, dout, cache, W_fixed):
        W, X = cache

        dW = X.T @ dout
        db = np.sum(dout, axis=0).reshape(1, -1) # db_1xn
        
#         dX = dout @ W.T # Backprop
        dX = dout @ W_fixed.T # fb alignment

        return dX, dW, db

    def train_forward(self, X, train):
        caches, ys = [], []
        
        # Input layer
        y, fc_cache = self.fc_forward(X=X, W=self.model[0]['W'], b=self.model[0]['b']) # X_1xD, y_1xc
        y, nl_cache = l.tanh_forward(X=y)
        if train:
            caches.append((fc_cache, nl_cache))
        ys.append(y) # ys[0]
        X = y.copy() # pass to the next layer
        
        # Hidden layers
        fc_caches, nl_caches, ys_L = [], [], []
        for layer in range(self.L):
            y, fc_cache = self.fc_forward(X=X, W=self.model[1][layer]['W'], b=self.model[1][layer]['b'])
            y, nl_cache = l.tanh_forward(X=y)
            ys_L.append(y) # ys[1][layer]
            X = y.copy() # pass to next layer
            if train:
                fc_caches.append(fc_cache)
                nl_caches.append(nl_cache)
        if train:
            caches.append((fc_caches, nl_caches)) # caches[1]            
        ys.append(ys_L) # ys[1]            
        
        # Output layer
        y, fc_cache = self.fc_forward(X=X, W=self.model[2]['W'], b=self.model[2]['b'])
        if train:
            caches.append(fc_cache)
        ys.append(y) # ys[2]

        return ys, caches # for backpropating the error

    def loss_function(self, y, y_train):
        
        loss = cross_entropy(y, y_train) # softmax is included
        dy = dcross_entropy(y, y_train) # dsoftmax is included
        
        return loss, dy
        
    def train_backward(self, dy, caches, ys):
        grads, ys_prev = self.grads, self.ys_prev # initialized by Zero in every iteration/epoch
        
        # Output layer
        fc_cache = caches[2]
        dX, dW, db = self.fc_backward(dout=dy, cache=fc_cache, W_fixed=self.W_fixed[2])
        dy = dX.copy()
        grads[2]['W'] = dW
        grads[2]['b'] = db

        # Hidden layer
        fc_caches, nl_caches = caches[1]
        for layer in reversed(range(self.L)):
#             dy *= ys[1][layer] - ys_prev[1][layer] # temporal diff instead of differentiable function
            dy = l.tanh_backward(cache=nl_caches[layer], dout=dy) # diffable function
            dX, dW, db = self.fc_backward(dout=dy, cache=fc_caches[layer], W_fixed=self.W_fixed[1][layer])
            dy = dX.copy()
            grads[1][layer]['W'] = dW
            grads[1][layer]['b'] = db
        
        # Input layer
        fc_cache, nl_cache = caches[0]
#         dy *= ys[0] - ys_prev[0] # temporal diff instead of differentiable function
        dy = l.tanh_backward(cache=nl_cache, dout=dy) # diffable function
        dX, dW, db = self.fc_backward(dout=dy, cache=fc_cache, W_fixed=self.W_fixed[0])
        grads[0]['W'] = dW
        grads[0]['b'] = db

        return dX, grads
    
    def test(self, X):
        ys_logit, _ = self.train_forward(X, train=False)
        y_logit = ys_logit[2] # last layer
        
        # if self.mode == 'classification':
        y_prob = l.softmax(y_logit) # for accuracy == acc
        y_pred = np.argmax(y_prob, axis=1) # for loss ==err
        
        return y_pred, y_logit
        
    def get_minibatch(self, X, y, minibatch_size, shuffle):
        minibatches = []

        if shuffle:
            X, y = skshuffle(X, y)

        for i in range(0, X.shape[0], minibatch_size):
            X_mini = X[i:i + minibatch_size]
            y_mini = y[i:i + minibatch_size]
            minibatches.append((X_mini, y_mini))

        return minibatches

    def sgd(self, train_set, val_set, alpha, mb_size, n_iter, print_after):
        X_train, y_train = train_set
        X_val, y_val = val_set

        # Epochs
        for iter in range(1, n_iter + 1):

            # Minibatches
            minibatches = self.get_minibatch(X_train, y_train, mb_size, shuffle=True)
            idx = np.random.randint(0, len(minibatches))
            X_mini, y_mini = minibatches[idx]
            
            # Train the model
            ys, caches = self.train_forward(X_mini, train=True)
            loss, dy = self.loss_function(ys[2], y_mini)
            _, grads = self.train_backward(dy, caches, ys) # ys[0], ys[1] and ys_prev are used for backprop
            self.ys_prev = ys # for next iteration or epoch learning dW and db
            self.losses['train'].append(loss)
            
            # Update the model for input layer
            for key in grads[0].keys():
                self.model[0][key] -= alpha * grads[0][key]

            # Update the model for the hidden layers
            for layer in range(self.L):
                for key in grads[1][layer].keys():
                    self.model[1][layer][key] -= alpha * grads[1][layer][key]

            # Update the model for output layer
            for key in grads[2].keys():
                self.model[2][key] -= alpha * grads[2][key]
                
            # Validate the updated model
            y_pred, y_logit = self.test(X_val)
            valid_loss, _ = self.loss_function(y_logit, y_val) # softmax is included in entropy loss function
            self.losses['valid'].append(valid_loss)
            valid_acc = np.mean(y_pred == y_val) # confusion matrix
            self.losses['valid_acc'].append(valid_acc)
            
            # Print the model info: loss & accuracy or err & acc
            if iter % print_after == 0:
                print('Iter-{} train loss: {:.4f} valid loss: {:.4f}, valid accuracy: {:.4f}'.format(
                    iter, loss, valid_loss, valid_acc))

        # Test the final model
        y_pred, y_logit = nn.test(X_test)
        loss, _ = self.loss_function(y_logit, y_test) # softmax is included in entropy loss function
        acc = np.mean(y_pred == y_test)
        print('Last iteration - Test accuracy mean: {:.4f}, std: {:.4f}, loss: {:.4f}'.format(
            acc.mean(), acc.std(), loss))

In [ ]:
# Hyper-parameters
n_iter = 1000000 # number of epochs
alpha = 1e-3 # learning_rate
mb_size = 50 # 2**10==1024 # width, timestep for sequential data or minibatch size
print_after = 100 # n_iter//10 # print loss for train, valid, and test
num_hidden_units = 32 # number of kernels/ filters in each layer
num_input_units = X_train.shape[1] # noise added at the input lavel as input noise we can use dX or for more improvement
num_output_units = y_train.max() + 1 # number of classes in this classification problem
num_layers = 2 # depth 

# Build the model/NN and learn it: running session.
nn = FFNN(C=num_output_units, D=num_input_units, H=num_hidden_units, L=num_layers)

nn.sgd(train_set=(X_train, y_train), val_set=(X_val, y_val), mb_size=mb_size, alpha=alpha, 
           n_iter=n_iter, print_after=print_after)

Iter-100 train loss: 2.3097 valid loss: 2.3107, valid accuracy: 0.0802
Iter-200 train loss: 2.3105 valid loss: 2.3123, valid accuracy: 0.0812
Iter-300 train loss: 2.3117 valid loss: 2.3137, valid accuracy: 0.0794
Iter-400 train loss: 2.3183 valid loss: 2.3149, valid accuracy: 0.0798
Iter-500 train loss: 2.3100 valid loss: 2.3159, valid accuracy: 0.0792
Iter-600 train loss: 2.3147 valid loss: 2.3165, valid accuracy: 0.0796
Iter-700 train loss: 2.3244 valid loss: 2.3167, valid accuracy: 0.0794
Iter-800 train loss: 2.3077 valid loss: 2.3166, valid accuracy: 0.0806
Iter-900 train loss: 2.3267 valid loss: 2.3158, valid accuracy: 0.0836
Iter-1000 train loss: 2.3127 valid loss: 2.3146, valid accuracy: 0.0854
Iter-1100 train loss: 2.3310 valid loss: 2.3131, valid accuracy: 0.0862
Iter-1200 train loss: 2.3438 valid loss: 2.3109, valid accuracy: 0.0872
Iter-1300 train loss: 2.2932 valid loss: 2.3082, valid accuracy: 0.0890
Iter-1400 train loss: 2.2944 valid loss: 2.3050, valid accuracy: 0.0906
I

Iter-11500 train loss: 1.3247 valid loss: 1.2044, valid accuracy: 0.6922
Iter-11600 train loss: 1.3651 valid loss: 1.1956, valid accuracy: 0.6960
Iter-11700 train loss: 1.1599 valid loss: 1.1867, valid accuracy: 0.6964
Iter-11800 train loss: 1.0998 valid loss: 1.1777, valid accuracy: 0.6988
Iter-11900 train loss: 1.0984 valid loss: 1.1693, valid accuracy: 0.7012
Iter-12000 train loss: 1.1258 valid loss: 1.1610, valid accuracy: 0.7036
Iter-12100 train loss: 1.1328 valid loss: 1.1527, valid accuracy: 0.7074
Iter-12200 train loss: 1.0046 valid loss: 1.1446, valid accuracy: 0.7100
Iter-12300 train loss: 1.2347 valid loss: 1.1365, valid accuracy: 0.7122
Iter-12400 train loss: 1.2578 valid loss: 1.1288, valid accuracy: 0.7130
Iter-12500 train loss: 1.0239 valid loss: 1.1210, valid accuracy: 0.7132
Iter-12600 train loss: 1.1510 valid loss: 1.1133, valid accuracy: 0.7160
Iter-12700 train loss: 1.0684 valid loss: 1.1057, valid accuracy: 0.7178
Iter-12800 train loss: 1.0345 valid loss: 1.0983, v

Iter-22800 train loss: 0.7429 valid loss: 0.7031, valid accuracy: 0.8172
Iter-22900 train loss: 0.8107 valid loss: 0.7010, valid accuracy: 0.8182
Iter-23000 train loss: 0.6681 valid loss: 0.6989, valid accuracy: 0.8196
Iter-23100 train loss: 0.6926 valid loss: 0.6969, valid accuracy: 0.8206
Iter-23200 train loss: 0.7128 valid loss: 0.6950, valid accuracy: 0.8212
Iter-23300 train loss: 0.6668 valid loss: 0.6930, valid accuracy: 0.8222
Iter-23400 train loss: 0.8144 valid loss: 0.6910, valid accuracy: 0.8224
Iter-23500 train loss: 0.6615 valid loss: 0.6891, valid accuracy: 0.8232
Iter-23600 train loss: 0.7539 valid loss: 0.6871, valid accuracy: 0.8230
Iter-23700 train loss: 0.6680 valid loss: 0.6851, valid accuracy: 0.8238
Iter-23800 train loss: 0.7885 valid loss: 0.6833, valid accuracy: 0.8244
Iter-23900 train loss: 0.7384 valid loss: 0.6814, valid accuracy: 0.8254
Iter-24000 train loss: 0.7731 valid loss: 0.6795, valid accuracy: 0.8258
Iter-24100 train loss: 0.4166 valid loss: 0.6776, v

Iter-34100 train loss: 0.7009 valid loss: 0.5455, valid accuracy: 0.8584
Iter-34200 train loss: 0.6587 valid loss: 0.5445, valid accuracy: 0.8588
Iter-34300 train loss: 0.6210 valid loss: 0.5436, valid accuracy: 0.8588
Iter-34400 train loss: 0.5044 valid loss: 0.5426, valid accuracy: 0.8588
Iter-34500 train loss: 0.7957 valid loss: 0.5417, valid accuracy: 0.8588
Iter-34600 train loss: 0.8407 valid loss: 0.5408, valid accuracy: 0.8590
Iter-34700 train loss: 0.4476 valid loss: 0.5400, valid accuracy: 0.8588
Iter-34800 train loss: 0.5602 valid loss: 0.5391, valid accuracy: 0.8588
Iter-34900 train loss: 0.6028 valid loss: 0.5382, valid accuracy: 0.8590
Iter-35000 train loss: 0.5992 valid loss: 0.5374, valid accuracy: 0.8594
Iter-35100 train loss: 0.4778 valid loss: 0.5365, valid accuracy: 0.8598
Iter-35200 train loss: 0.5871 valid loss: 0.5356, valid accuracy: 0.8596
Iter-35300 train loss: 0.5205 valid loss: 0.5347, valid accuracy: 0.8596
Iter-35400 train loss: 0.6746 valid loss: 0.5339, v

Iter-45400 train loss: 0.6291 valid loss: 0.4667, valid accuracy: 0.8742
Iter-45500 train loss: 0.4978 valid loss: 0.4663, valid accuracy: 0.8746
Iter-45600 train loss: 0.7108 valid loss: 0.4658, valid accuracy: 0.8742
Iter-45700 train loss: 0.3638 valid loss: 0.4652, valid accuracy: 0.8746
Iter-45800 train loss: 0.4252 valid loss: 0.4647, valid accuracy: 0.8748
Iter-45900 train loss: 0.3812 valid loss: 0.4643, valid accuracy: 0.8750
Iter-46000 train loss: 0.6263 valid loss: 0.4638, valid accuracy: 0.8752
Iter-46100 train loss: 0.4347 valid loss: 0.4632, valid accuracy: 0.8748
Iter-46200 train loss: 0.3484 valid loss: 0.4627, valid accuracy: 0.8750
Iter-46300 train loss: 0.6050 valid loss: 0.4622, valid accuracy: 0.8752
Iter-46400 train loss: 0.3885 valid loss: 0.4617, valid accuracy: 0.8752
Iter-46500 train loss: 0.5451 valid loss: 0.4612, valid accuracy: 0.8756
Iter-46600 train loss: 0.6189 valid loss: 0.4608, valid accuracy: 0.8758
Iter-46700 train loss: 0.5679 valid loss: 0.4602, v

...

Iter-312700 train loss: 0.2615 valid loss: 0.2563, valid accuracy: 0.9230
Iter-312800 train loss: 0.4597 valid loss: 0.2562, valid accuracy: 0.9230
Iter-312900 train loss: 0.3958 valid loss: 0.2562, valid accuracy: 0.9230
Iter-313000 train loss: 0.2717 valid loss: 0.2561, valid accuracy: 0.9232
Iter-313100 train loss: 0.1722 valid loss: 0.2561, valid accuracy: 0.9230
Iter-313200 train loss: 0.2872 valid loss: 0.2561, valid accuracy: 0.9230
Iter-313300 train loss: 0.1965 valid loss: 0.2561, valid accuracy: 0.9230
Iter-313400 train loss: 0.2116 valid loss: 0.2561, valid accuracy: 0.9230
Iter-313500 train loss: 0.2055 valid loss: 0.2561, valid accuracy: 0.9228
Iter-313600 train loss: 0.1734 valid loss: 0.2560, valid accuracy: 0.9230
Iter-313700 train loss: 0.0763 valid loss: 0.2560, valid accuracy: 0.9232
Iter-313800 train loss: 0.3362 valid loss: 0.2560, valid accuracy: 0.9232
Iter-313900 train loss: 0.2380 valid loss: 0.2560, valid accuracy: 0.9232
Iter-314000 train loss: 0.1179 valid l

Iter-323800 train loss: 0.1572 valid loss: 0.2542, valid accuracy: 0.9230
Iter-323900 train loss: 0.2812 valid loss: 0.2541, valid accuracy: 0.9232
Iter-324000 train loss: 0.4008 valid loss: 0.2541, valid accuracy: 0.9232
Iter-324100 train loss: 0.2962 valid loss: 0.2541, valid accuracy: 0.9232
Iter-324200 train loss: 0.1809 valid loss: 0.2541, valid accuracy: 0.9232
Iter-324300 train loss: 0.1518 valid loss: 0.2541, valid accuracy: 0.9232
Iter-324400 train loss: 0.1790 valid loss: 0.2541, valid accuracy: 0.9232
Iter-324500 train loss: 0.2614 valid loss: 0.2541, valid accuracy: 0.9232
Iter-324600 train loss: 0.5588 valid loss: 0.2541, valid accuracy: 0.9230
Iter-324700 train loss: 0.2371 valid loss: 0.2540, valid accuracy: 0.9230
Iter-324800 train loss: 0.1210 valid loss: 0.2540, valid accuracy: 0.9226
Iter-324900 train loss: 0.3828 valid loss: 0.2541, valid accuracy: 0.9230
Iter-325000 train loss: 0.4259 valid loss: 0.2540, valid accuracy: 0.9230
Iter-325100 train loss: 0.3500 valid l

Iter-334900 train loss: 0.1981 valid loss: 0.2526, valid accuracy: 0.9236
Iter-335000 train loss: 0.1123 valid loss: 0.2526, valid accuracy: 0.9236
Iter-335100 train loss: 0.1545 valid loss: 0.2525, valid accuracy: 0.9234
Iter-335200 train loss: 0.3457 valid loss: 0.2525, valid accuracy: 0.9232
Iter-335300 train loss: 0.0750 valid loss: 0.2525, valid accuracy: 0.9234
Iter-335400 train loss: 0.1879 valid loss: 0.2525, valid accuracy: 0.9232
Iter-335500 train loss: 0.3408 valid loss: 0.2524, valid accuracy: 0.9234
Iter-335600 train loss: 0.2374 valid loss: 0.2524, valid accuracy: 0.9234
Iter-335700 train loss: 0.2911 valid loss: 0.2524, valid accuracy: 0.9236
Iter-335800 train loss: 0.2964 valid loss: 0.2523, valid accuracy: 0.9236
Iter-335900 train loss: 0.2653 valid loss: 0.2523, valid accuracy: 0.9236
Iter-336000 train loss: 0.3468 valid loss: 0.2523, valid accuracy: 0.9236
Iter-336100 train loss: 0.1577 valid loss: 0.2523, valid accuracy: 0.9236
Iter-336200 train loss: 0.1575 valid l

Iter-346000 train loss: 0.1277 valid loss: 0.2507, valid accuracy: 0.9236
Iter-346100 train loss: 0.1422 valid loss: 0.2507, valid accuracy: 0.9234
Iter-346200 train loss: 0.1982 valid loss: 0.2506, valid accuracy: 0.9234
Iter-346300 train loss: 0.2514 valid loss: 0.2506, valid accuracy: 0.9236
Iter-346400 train loss: 0.4354 valid loss: 0.2506, valid accuracy: 0.9236
Iter-346500 train loss: 0.4014 valid loss: 0.2506, valid accuracy: 0.9236
Iter-346600 train loss: 0.2637 valid loss: 0.2506, valid accuracy: 0.9236
Iter-346700 train loss: 0.1741 valid loss: 0.2506, valid accuracy: 0.9234
Iter-346800 train loss: 0.3132 valid loss: 0.2506, valid accuracy: 0.9236
Iter-346900 train loss: 0.1209 valid loss: 0.2506, valid accuracy: 0.9234
Iter-347000 train loss: 0.4387 valid loss: 0.2506, valid accuracy: 0.9234
Iter-347100 train loss: 0.4257 valid loss: 0.2505, valid accuracy: 0.9238
Iter-347200 train loss: 0.3835 valid loss: 0.2505, valid accuracy: 0.9238
Iter-347300 train loss: 0.1650 valid l

Iter-357100 train loss: 0.2311 valid loss: 0.2490, valid accuracy: 0.9240
Iter-357200 train loss: 0.4596 valid loss: 0.2490, valid accuracy: 0.9240
Iter-357300 train loss: 0.3490 valid loss: 0.2489, valid accuracy: 0.9238
Iter-357400 train loss: 0.1650 valid loss: 0.2489, valid accuracy: 0.9238
Iter-357500 train loss: 0.0941 valid loss: 0.2489, valid accuracy: 0.9240
Iter-357600 train loss: 0.0850 valid loss: 0.2489, valid accuracy: 0.9240
Iter-357700 train loss: 0.1758 valid loss: 0.2489, valid accuracy: 0.9240
Iter-357800 train loss: 0.4211 valid loss: 0.2488, valid accuracy: 0.9238
Iter-357900 train loss: 0.3193 valid loss: 0.2488, valid accuracy: 0.9238
Iter-358000 train loss: 0.2229 valid loss: 0.2488, valid accuracy: 0.9238
Iter-358100 train loss: 0.2831 valid loss: 0.2488, valid accuracy: 0.9240
Iter-358200 train loss: 0.3681 valid loss: 0.2488, valid accuracy: 0.9236
Iter-358300 train loss: 0.2984 valid loss: 0.2487, valid accuracy: 0.9238
Iter-358400 train loss: 0.3480 valid l

In [ ]:
# # Display the learning curve and losses for training, validation, and testing
# %matplotlib inline
# %config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt

plt.plot(nn.losses['train'], label='Train loss')
plt.plot(nn.losses['valid'], label='Valid loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(nn.losses['valid_acc'], label='Valid accuracy')
plt.legend()
plt.show()